# AI Základy - Hodina 23: Neurčitost a predikce

## Obsah:
1. **Co je to nejistota v AI?**
2. **Pravděpodobnostní modely**
3. **Bayesovo uvažování**
4. **Predikční modely a práce s nejistotou**
5. **Praktické aplikace**
6. **Domácí úkol**

## 1. Co je to nejistota?

### 1.1 Zdroje nejistoty v AI

Nejistota v AI pochází z několika zdrojů:

1. **Epistemická nejistota** - nedostatek znalostí o systému
2. **Aleatorická nejistota** - inherentní náhodnost v datech
3. **Chybějící data** - neúplné informace
4. **Šum v datech** - měřicí chyby, nepřesnosti
5. **Modelová nejistota** - nejistota v parametrech modelu

In [ ]:
# Import potřebných knihoven
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LogisticRegression, BayesianRidge
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report
from sklearn.naive_bayes import GaussianNB
import warnings
warnings.filterwarnings('ignore')

# Nastavení vizualizace
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

# Pro interaktivní aplikace
import gradio as gr

### 1.2 Vizualizace různých typů nejistoty

In [ ]:
# Demonstrace různých typů nejistoty
np.random.seed(42)

# Vytvoření dat s různými typy nejistoty
x = np.linspace(0, 10, 100)

# 1. Data bez nejistoty (ideální případ)
y_ideal = 2 * x + 3

# 2. Aleatorická nejistota (náhodný šum)
noise = np.random.normal(0, 2, size=x.shape)
y_aleatory = y_ideal + noise

# 3. Epistemická nejistota (systematická chyba)
y_epistemic = 2.5 * x + 2  # Špatné parametry modelu

# 4. Kombinovaná nejistota
y_combined = y_epistemic + noise

# Vizualizace
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

# Graf 1: Ideální data
axes[0].plot(x, y_ideal, 'b-', linewidth=2, label='Skutečná funkce')
axes[0].set_title('Ideální data (bez nejistoty)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Graf 2: Aleatorická nejistota
axes[1].scatter(x, y_aleatory, alpha=0.6, s=30, label='Data s šumem')
axes[1].plot(x, y_ideal, 'r--', linewidth=2, label='Skutečná funkce')
axes[1].set_title('Aleatorická nejistota (náhodný šum)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Graf 3: Epistemická nejistota
axes[2].plot(x, y_ideal, 'b-', linewidth=2, label='Skutečná funkce')
axes[2].plot(x, y_epistemic, 'g-', linewidth=2, label='Špatný model')
axes[2].fill_between(x, y_ideal, y_epistemic, alpha=0.3, color='red', label='Modelová chyba')
axes[2].set_title('Epistemická nejistota (chyba modelu)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('X')
axes[2].set_ylabel('Y')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Graf 4: Kombinovaná nejistota
axes[3].scatter(x, y_combined, alpha=0.6, s=30, color='purple', label='Data s kombinovanou nejistotou')
axes[3].plot(x, y_ideal, 'b--', linewidth=2, label='Skutečná funkce')
axes[3].plot(x, y_epistemic, 'g--', linewidth=2, label='Špatný model')
axes[3].set_title('Kombinovaná nejistota', fontsize=14, fontweight='bold')
axes[3].set_xlabel('X')
axes[3].set_ylabel('Y')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("=" * 60)
print("TYPY NEJISTOTY V AI:")
print("=" * 60)
print("\n1. ALEATORICKÁ NEJISTOTA:")
print("   - Přirozená náhodnost v datech")
print("   - Nelze ji odstranit více daty")
print("   - Příklad: házení kostkou, kvantové jevy")
print("\n2. EPISTEMICKÁ NEJISTOTA:")
print("   - Nedostatek znalostí o systému")
print("   - Lze snížit získáním více dat nebo lepším modelem")
print("   - Příklad: neznámé parametry modelu")

### 1.3 Práce s chybějícími daty

In [ ]:
# Vytvoření datasetu s chybějícími hodnotami
np.random.seed(42)

# Generování syntetických dat
n_samples = 1000
n_features = 5

# Vytvoření kompletních dat
X_complete = np.random.randn(n_samples, n_features)
# Lineární závislost s šumem
y_complete = 2*X_complete[:, 0] - 1.5*X_complete[:, 1] + 0.5*X_complete[:, 2] + np.random.randn(n_samples)*0.5

# Přidání chybějících hodnot (20% náhodně)
missing_mask = np.random.random((n_samples, n_features)) < 0.2
X_missing = X_complete.copy()
X_missing[missing_mask] = np.nan

# Vytvoření DataFrame pro lepší práci
feature_names = [f'Feature_{i+1}' for i in range(n_features)]
df_complete = pd.DataFrame(X_complete, columns=feature_names)
df_missing = pd.DataFrame(X_missing, columns=feature_names)
df_missing['Target'] = y_complete

# Vizualizace chybějících dat
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Heatmapa chybějících hodnot
missing_data = df_missing.isnull()
sns.heatmap(missing_data.head(50), cbar=True, yticklabels=False, 
            cmap='viridis', ax=ax1)
ax1.set_title('Vzor chybějících dat (prvních 50 vzorků)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Proměnné')

# Sloupcový graf procenta chybějících hodnot
missing_percent = (missing_data.sum() / len(df_missing)) * 100
missing_percent.plot(kind='bar', ax=ax2, color='coral')
ax2.set_title('Procento chybějících hodnot pro každou proměnnou', fontsize=14, fontweight='bold')
ax2.set_xlabel('Proměnné')
ax2.set_ylabel('Procento chybějících hodnot')
ax2.set_ylim(0, 30)

# Přidání hodnot na graf
for i, v in enumerate(missing_percent):
    ax2.text(i, v + 0.5, f'{v:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\nCelkový počet chybějících hodnot: {missing_data.sum().sum()}")
print(f"Procento chybějících dat: {(missing_data.sum().sum() / (n_samples * n_features)) * 100:.1f}%")

## 2. Pravděpodobnostní modely

### 2.1 Proč pravděpodobnostní přístup?

Pravděpodobnostní modely nám umožňují:
- Kvantifikovat nejistotu v předpovědích
- Rozhodovat se na základě rizika
- Kombinovat různé zdroje informací
- Aktualizovat přesvědčení s novými daty

In [ ]:
# Porovnání deterministického a pravděpodobnostního modelu
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

# Vytvoření nelineárních dat s různou nejistotou
np.random.seed(42)
X_train = np.sort(np.random.uniform(0, 10, 30))
y_train = np.sin(X_train) + 0.1 * X_train + np.random.normal(0, 0.2, X_train.shape)

# Oblasti s různou hustotou dat (různá nejistota)
mask_sparse = (X_train > 6) & (X_train < 8)
X_train = X_train[~mask_sparse]
y_train = y_train[~mask_sparse]

X_test = np.linspace(0, 10, 200).reshape(-1, 1)
X_train = X_train.reshape(-1, 1)

# 1. Deterministický model (Linear Regression)
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

# 2. Pravděpodobnostní model (Gaussian Process)
kernel = ConstantKernel(1.0) * RBF(length_scale=1.0) + WhiteKernel(noise_level=0.1)
gp_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1)
gp_model.fit(X_train, y_train)
y_pred_gp, y_std_gp = gp_model.predict(X_test, return_std=True)

# Vizualizace
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# Graf 1: Deterministický model
ax1.scatter(X_train, y_train, c='red', s=50, zorder=10, 
           edgecolors='black', label='Trénovací data')
ax1.plot(X_test, y_pred_lr, 'b-', linewidth=2, label='Lineární regrese')
ax1.set_title('Deterministický model (Lineární regrese)', fontsize=16, fontweight='bold')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.axvspan(6, 8, alpha=0.2, color='gray', label='Oblast bez dat')

# Graf 2: Pravděpodobnostní model
ax2.scatter(X_train, y_train, c='red', s=50, zorder=10, 
           edgecolors='black', label='Trénovací data')
ax2.plot(X_test, y_pred_gp, 'b-', linewidth=2, label='Gaussovský proces (střední hodnota)')
ax2.fill_between(X_test.ravel(), 
                y_pred_gp - 2*y_std_gp, 
                y_pred_gp + 2*y_std_gp, 
                alpha=0.3, color='blue', label='95% interval spolehlivosti')
ax2.set_title('Pravděpodobnostní model (Gaussovský proces)', fontsize=16, fontweight='bold')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.axvspan(6, 8, alpha=0.2, color='gray')

plt.tight_layout()
plt.show()

print("=" * 70)
print("KLÍČOVÉ ROZDÍLY:")
print("=" * 70)
print("\n1. DETERMINISTICKÝ MODEL:")
print("   - Poskytuje pouze bodovou předpověď")
print("   - Neříká nic o spolehlivosti předpovědi")
print("   - Stejná jistota všude (i v oblastech bez dat)")
print("\n2. PRAVDĚPODOBNOSTNÍ MODEL:")
print("   - Poskytuje předpověď s intervalem spolehlivosti")
print("   - Nejistota se zvyšuje v oblastech bez dat")
print("   - Umožňuje kvantifikovat riziko rozhodnutí")

### 2.2 Pravděpodobnostní klasifikace

In [ ]:
# Vytvoření klasifikačního problému s překrývajícími se třídami
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

# Generování dat
X_class, y_class = make_classification(n_samples=500, n_features=2, n_redundant=0,
                                       n_informative=2, n_clusters_per_class=1,
                                       flip_y=0.1, class_sep=0.5, random_state=42)

# Rozdělení na trénovací a testovací
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_class, y_class, 
                                                            test_size=0.3, random_state=42)

# Trénování modelu
log_reg = LogisticRegression()
log_reg.fit(X_train_c, y_train_c)

# Předpovědi s pravděpodobnostmi
y_pred_proba = log_reg.predict_proba(X_test_c)
y_pred_class = log_reg.predict(X_test_c)

# Vytvoření mřížky pro vizualizaci rozhodovací hranice
h = 0.02
x_min, x_max = X_class[:, 0].min() - 1, X_class[:, 0].max() + 1
y_min, y_max = X_class[:, 1].min() - 1, X_class[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z_proba = log_reg.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
Z_proba = Z_proba.reshape(xx.shape)

# Vizualizace
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Graf 1: Trénovací data a rozhodovací hranice
contour = axes[0].contourf(xx, yy, Z_proba, levels=20, cmap='RdBu_r', alpha=0.8)
axes[0].scatter(X_train_c[:, 0], X_train_c[:, 1], c=y_train_c, 
               cmap='RdBu', edgecolors='black', s=50)
axes[0].set_title('Pravděpodobnostní mapa klasifikace', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
cbar = plt.colorbar(contour, ax=axes[0])
cbar.set_label('P(Class 1)', rotation=270, labelpad=20)

# Graf 2: Nejistota v předpovědích
uncertainty = np.minimum(Z_proba, 1-Z_proba) * 2  # Nejvyšší nejistota když P=0.5
im = axes[1].contourf(xx, yy, uncertainty, levels=20, cmap='viridis')
axes[1].scatter(X_test_c[:, 0], X_test_c[:, 1], c=y_test_c, 
               cmap='RdBu', edgecolors='black', s=50)
axes[1].set_title('Mapa nejistoty předpovědí', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
cbar2 = plt.colorbar(im, ax=axes[1])
cbar2.set_label('Nejistota', rotation=270, labelpad=20)

# Graf 3: Histogram pravděpodobností
axes[2].hist(y_pred_proba[:, 1], bins=30, alpha=0.7, color='purple', edgecolor='black')
axes[2].axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Rozhodovací práh')
axes[2].set_title('Distribuce předpovězených pravděpodobností', fontsize=14, fontweight='bold')
axes[2].set_xlabel('P(Class 1)')
axes[2].set_ylabel('Počet vzorků')
axes[2].legend()

plt.tight_layout()
plt.show()

# Analýza nejistých předpovědí
uncertain_mask = (y_pred_proba[:, 1] > 0.4) & (y_pred_proba[:, 1] < 0.6)
print(f"\nPočet nejistých předpovědí (0.4 < P < 0.6): {uncertain_mask.sum()} z {len(y_test_c)}")
print(f"Procento nejistých předpovědí: {uncertain_mask.sum()/len(y_test_c)*100:.1f}%")
print(f"\nPřesnost na jistých předpovědích: {accuracy_score(y_test_c[~uncertain_mask], y_pred_class[~uncertain_mask]):.3f}")
print(f"Přesnost na nejistých předpovědích: {accuracy_score(y_test_c[uncertain_mask], y_pred_class[uncertain_mask]):.3f}")

## 3. Bayesovo uvažování

### 3.1 Bayesova věta v kontextu AI

Bayesova věta nám umožňuje aktualizovat naše přesvědčení na základě nových důkazů:

$$P(H|E) = \frac{P(E|H) \cdot P(H)}{P(E)}$$

kde:
- P(H|E) - posteriorní pravděpodobnost (po pozorování důkazů)
- P(H) - apriorní pravděpodobnost (před pozorováním)
- P(E|H) - věrohodnost (likelihood)
- P(E) - marginální pravděpodobnost důkazů

In [ ]:
# Příklad: Bayesovská aktualizace pro detekci spamu

def bayesian_spam_detector():
    print("=" * 70)
    print("BAYESOVSKÝ SPAM DETEKTOR")
    print("=" * 70)
    
    # Počáteční pravděpodobnosti (prior)
    P_spam_prior = 0.3  # 30% emailů je spam
    P_ham_prior = 0.7   # 70% emailů je ham (ne-spam)
    
    # Pravděpodobnosti slov v spamu a hamu
    spam_words = {
        'zdarma': 0.9,
        'výhra': 0.8,
        'klikněte': 0.7,
        'ihned': 0.6,
        'práce': 0.2,
        'schůzka': 0.1
    }
    
    ham_words = {
        'zdarma': 0.1,
        'výhra': 0.05,
        'klikněte': 0.1,
        'ihned': 0.2,
        'práce': 0.7,
        'schůzka': 0.8
    }
    
    # Testovací emaily
    test_emails = [
        ['zdarma', 'výhra', 'klikněte'],
        ['práce', 'schůzka'],
        ['ihned', 'klikněte', 'zdarma'],
        ['schůzka', 'práce', 'ihned']
    ]
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.ravel()
    
    for idx, email_words in enumerate(test_emails):
        # Výpočet likelihood
        P_words_given_spam = 1
        P_words_given_ham = 1
        
        for word in email_words:
            P_words_given_spam *= spam_words.get(word, 0.01)
            P_words_given_ham *= ham_words.get(word, 0.01)
        
        # Bayesova věta
        P_words = P_words_given_spam * P_spam_prior + P_words_given_ham * P_ham_prior
        P_spam_posterior = (P_words_given_spam * P_spam_prior) / P_words
        P_ham_posterior = (P_words_given_ham * P_ham_prior) / P_words
        
        # Vizualizace
        ax = axes[idx]
        
        # Sloupcový graf
        categories = ['Prior', 'Posterior']
        spam_probs = [P_spam_prior, P_spam_posterior]
        ham_probs = [P_ham_prior, P_ham_posterior]
        
        x = np.arange(len(categories))
        width = 0.35
        
        bars1 = ax.bar(x - width/2, spam_probs, width, label='Spam', color='red', alpha=0.7)
        bars2 = ax.bar(x + width/2, ham_probs, width, label='Ham', color='green', alpha=0.7)
        
        ax.set_ylabel('Pravděpodobnost')
        ax.set_title(f'Email {idx+1}: {" + ".join(email_words)}', fontsize=12, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(categories)
        ax.legend()
        ax.set_ylim(0, 1)
        
        # Přidání hodnot
        for bars, values in [(bars1, spam_probs), (bars2, ham_probs)]:
            for bar, val in zip(bars, values):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{val:.3f}', ha='center', va='bottom')
        
        # Výsledek klasifikace
        classification = "SPAM" if P_spam_posterior > 0.5 else "HAM"
        color = 'red' if classification == "SPAM" else 'green'
        ax.text(0.5, 0.85, f'Klasifikace: {classification}', 
               transform=ax.transAxes, ha='center', fontsize=14, 
               fontweight='bold', color=color,
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.suptitle('Bayesovská aktualizace pro detekci spamu', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\nVýsledky klasifikace:")
    for idx, email_words in enumerate(test_emails):
        print(f"Email {idx+1} ({', '.join(email_words)}): ", end="")
        
        # Přepočet pro výpis
        P_words_given_spam = 1
        P_words_given_ham = 1
        for word in email_words:
            P_words_given_spam *= spam_words.get(word, 0.01)
            P_words_given_ham *= ham_words.get(word, 0.01)
        P_words = P_words_given_spam * P_spam_prior + P_words_given_ham * P_ham_prior
        P_spam_posterior = (P_words_given_spam * P_spam_prior) / P_words
        
        classification = "SPAM" if P_spam_posterior > 0.5 else "HAM"
        print(f"{classification} (P(spam) = {P_spam_posterior:.3f})")

bayesian_spam_detector()

### 3.2 Sekvenční Bayesovská aktualizace

In [ ]:
# Demonstrace postupné aktualizace přesvědčení
def sequential_bayesian_update():
    # Scénář: Testování, zda je mince férová
    # H1: Mince je férová (P(hlava) = 0.5)
    # H2: Mince je zkreslená (P(hlava) = 0.7)
    
    # Počáteční přesvědčení (uniformní prior)
    prior_fair = 0.5
    prior_biased = 0.5
    
    # Pravděpodobnosti pro každou hypotézu
    p_heads_fair = 0.5
    p_heads_biased = 0.7
    
    # Simulace hodů
    np.random.seed(42)
    n_flips = 50
    # Simulujeme zkreslenou minci
    flips = np.random.random(n_flips) < 0.7  # True = hlava
    
    # Ukládání posteriorů
    posteriors_fair = [prior_fair]
    posteriors_biased = [prior_biased]
    
    # Sekvenční aktualizace
    for flip in flips:
        # Likelihood
        if flip:  # Hlava
            likelihood_fair = p_heads_fair
            likelihood_biased = p_heads_biased
        else:  # Orel
            likelihood_fair = 1 - p_heads_fair
            likelihood_biased = 1 - p_heads_biased
        
        # Aktualizace pomocí Bayesovy věty
        evidence = likelihood_fair * posteriors_fair[-1] + likelihood_biased * posteriors_biased[-1]
        
        posterior_fair = (likelihood_fair * posteriors_fair[-1]) / evidence
        posterior_biased = (likelihood_biased * posteriors_biased[-1]) / evidence
        
        posteriors_fair.append(posterior_fair)
        posteriors_biased.append(posterior_biased)
    
    # Vizualizace
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Graf 1: Vývoj posteriorních pravděpodobností
    x = range(len(posteriors_fair))
    ax1.plot(x, posteriors_fair, 'b-', linewidth=2, label='P(Férová mince)')
    ax1.plot(x, posteriors_biased, 'r-', linewidth=2, label='P(Zkreslená mince)')
    ax1.fill_between(x, 0, posteriors_fair, alpha=0.3, color='blue')
    ax1.fill_between(x, posteriors_fair, 1, alpha=0.3, color='red')
    ax1.set_xlabel('Počet hodů')
    ax1.set_ylabel('Posteriorní pravděpodobnost')
    ax1.set_title('Bayesovská aktualizace přesvědčení o férovosti mince', 
                 fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1)
    
    # Graf 2: Kumulativní počet hlav
    cumsum_heads = np.cumsum(flips)
    cumsum_total = np.arange(1, len(flips) + 1)
    observed_ratio = cumsum_heads / cumsum_total
    
    ax2.plot(cumsum_total, observed_ratio, 'g-', linewidth=2, 
            label='Pozorovaný poměr hlav')
    ax2.axhline(y=0.5, color='blue', linestyle='--', linewidth=2, 
               label='Očekáváno pro férovou minci')
    ax2.axhline(y=0.7, color='red', linestyle='--', linewidth=2, 
               label='Očekáváno pro zkreslenou minci')
    ax2.set_xlabel('Počet hodů')
    ax2.set_ylabel('Poměr hlav')
    ax2.set_title('Pozorovaný poměr hlav v čase', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0.3, 0.9)
    
    plt.tight_layout()
    plt.show()
    
    # Finální výsledky
    print(f"\nPo {n_flips} hodech:")
    print(f"Počet hlav: {sum(flips)} ({sum(flips)/n_flips:.1%})")
    print(f"\nFinální posteriorní pravděpodobnosti:")
    print(f"P(Férová mince | data) = {posteriors_fair[-1]:.4f}")
    print(f"P(Zkreslená mince | data) = {posteriors_biased[-1]:.4f}")
    
    if posteriors_biased[-1] > posteriors_fair[-1]:
        print("\nZávěr: Mince je pravděpodobně ZKRESLENÁ")
    else:
        print("\nZávěr: Mince je pravděpodobně FÉROVÁ")

sequential_bayesian_update()

## 4. Predikční modely s nejistotou

### 4.1 Práce s chybějícími daty v predikčních modelech

In [ ]:
# Porovnání různých strategií pro práci s chybějícími daty
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Použijeme dříve vytvořená data s chybějícími hodnotami
X_train_missing, X_test_missing, y_train, y_test = train_test_split(
    X_missing, y_complete, test_size=0.3, random_state=42
)

# Různé strategie imputace
strategies = {
    'Odstranění řádků': None,
    'Průměr': SimpleImputer(strategy='mean'),
    'Medián': SimpleImputer(strategy='median'),
    'KNN Imputer': KNNImputer(n_neighbors=5),
    'Iterativní': IterativeImputer(random_state=42)
}

results = {}

for name, imputer in strategies.items():
    if name == 'Odstranění řádků':
        # Odstranění řádků s chybějícími hodnotami
        mask_train = ~np.isnan(X_train_missing).any(axis=1)
        mask_test = ~np.isnan(X_test_missing).any(axis=1)
        X_train_imp = X_train_missing[mask_train]
        X_test_imp = X_test_missing[mask_test]
        y_train_imp = y_train[mask_train]
        y_test_imp = y_test[mask_test]
    else:
        # Imputace
        X_train_imp = imputer.fit_transform(X_train_missing)
        X_test_imp = imputer.transform(X_test_missing)
        y_train_imp = y_train
        y_test_imp = y_test
    
    # Trénování modelu
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Pro regresi převedeme na klasifikaci
    y_train_class = (y_train_imp > np.median(y_train_imp)).astype(int)
    y_test_class = (y_test_imp > np.median(y_test_imp)).astype(int)
    
    model.fit(X_train_imp, y_train_class)
    
    # Predikce s pravděpodobnostmi
    y_pred_proba = model.predict_proba(X_test_imp)
    y_pred = model.predict(X_test_imp)
    
    # Uložení výsledků
    accuracy = accuracy_score(y_test_class, y_pred)
    results[name] = {
        'accuracy': accuracy,
        'train_size': len(X_train_imp),
        'test_size': len(X_test_imp),
        'probabilities': y_pred_proba
    }

# Vizualizace výsledků
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Graf 1: Porovnání přesností
names = list(results.keys())
accuracies = [results[name]['accuracy'] for name in names]
colors = plt.cm.viridis(np.linspace(0, 1, len(names)))

bars = ax1.bar(range(len(names)), accuracies, color=colors)
ax1.set_xticks(range(len(names)))
ax1.set_xticklabels(names, rotation=45, ha='right')
ax1.set_ylabel('Přesnost')
ax1.set_title('Porovnání strategií pro chybějící data', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 1)

# Přidání hodnot
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
            f'{acc:.3f}', ha='center', va='bottom')

# Graf 2: Velikost datasetů po zpracování
train_sizes = [results[name]['train_size'] for name in names]
test_sizes = [results[name]['test_size'] for name in names]

x = np.arange(len(names))
width = 0.35

bars1 = ax2.bar(x - width/2, train_sizes, width, label='Trénovací', color='lightblue')
bars2 = ax2.bar(x + width/2, test_sizes, width, label='Testovací', color='lightcoral')

ax2.set_ylabel('Počet vzorků')
ax2.set_title('Velikost datasetu po zpracování', fontsize=14, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(names, rotation=45, ha='right')
ax2.legend()

plt.tight_layout()
plt.show()

print("\nShrnutí výsledků:")
print("=" * 60)
for name in names:
    print(f"\n{name}:")
    print(f"  Přesnost: {results[name]['accuracy']:.3f}")
    print(f"  Trénovací vzorky: {results[name]['train_size']}")
    print(f"  Testovací vzorky: {results[name]['test_size']}")

### 4.2 Kvantifikace nejistoty v predikčních modelech

In [ ]:
# Ensemble metody pro kvantifikaci nejistoty
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Vytvoření nelineárních dat
np.random.seed(42)
n_samples = 200
X = np.sort(np.random.uniform(0, 10, n_samples))
y_true = np.sin(X) + 0.1 * X**2
noise = np.random.normal(0, 0.5, n_samples)
y = y_true + noise

# Přidání oblasti s větší nejistotou (méně dat)
mask = (X > 3) & (X < 5)
X_train = X[~mask].reshape(-1, 1)
y_train = y[~mask]

# Test data
X_test = np.linspace(0, 10, 300).reshape(-1, 1)

# Random Forest s více stromy pro lepší odhad nejistoty
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predikce - průměr přes všechny stromy
y_pred_mean = rf.predict(X_test)

# Získání predikcí z jednotlivých stromů
predictions_trees = np.array([tree.predict(X_test) for tree in rf.estimators_])

# Výpočet nejistoty (std přes stromy)
y_pred_std = np.std(predictions_trees, axis=0)

# Bayesovská ridge regrese pro porovnání
from sklearn.linear_model import BayesianRidge
br = BayesianRidge()
br.fit(X_train, y_train)
y_pred_br, y_std_br = br.predict(X_test, return_std=True)

# Vizualizace
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12))

# Graf 1: Random Forest s nejistotou
ax1.scatter(X_train, y_train, c='red', s=50, alpha=0.7, 
           edgecolors='black', label='Trénovací data')
ax1.plot(X_test, y_pred_mean, 'b-', linewidth=2, label='Random Forest predikce')
ax1.fill_between(X_test.ravel(), 
                y_pred_mean - 2*y_pred_std, 
                y_pred_mean + 2*y_pred_std, 
                alpha=0.3, color='blue', label='±2σ nejistota')
ax1.axvspan(3, 5, alpha=0.1, color='gray', label='Oblast s chybějícími daty')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_title('Random Forest - Kvantifikace nejistoty pomocí ensemble', 
             fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Graf 2: Bayesovská regrese
ax2.scatter(X_train, y_train, c='red', s=50, alpha=0.7, 
           edgecolors='black', label='Trénovací data')
ax2.plot(X_test, y_pred_br, 'g-', linewidth=2, label='Bayesovská ridge predikce')
ax2.fill_between(X_test.ravel(), 
                y_pred_br - 2*y_std_br, 
                y_pred_br + 2*y_std_br, 
                alpha=0.3, color='green', label='±2σ nejistota')
ax2.axvspan(3, 5, alpha=0.1, color='gray', label='Oblast s chybějícími daty')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_title('Bayesovská Ridge Regrese - Přirozená kvantifikace nejistoty', 
             fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analýza nejistoty v různých oblastech
regions = {
    'Oblast s daty (0-3)': (X_test.ravel() >= 0) & (X_test.ravel() <= 3),
    'Oblast bez dat (3-5)': (X_test.ravel() > 3) & (X_test.ravel() < 5),
    'Oblast s daty (5-10)': (X_test.ravel() >= 5) & (X_test.ravel() <= 10)
}

print("\nPrůměrná nejistota v různých oblastech:")
print("=" * 60)
for region_name, mask in regions.items():
    rf_uncertainty = np.mean(y_pred_std[mask])
    br_uncertainty = np.mean(y_std_br[mask])
    print(f"\n{region_name}:")
    print(f"  Random Forest σ: {rf_uncertainty:.3f}")
    print(f"  Bayesovská regrese σ: {br_uncertainty:.3f}")

## 5. Interaktivní aplikace - Simulátor nejistoty

In [ ]:
def uncertainty_simulator(model_type, noise_level, missing_data_percent, n_samples):
    """
    Interaktivní simulátor pro demonstraci vlivu různých faktorů na nejistotu
    """
    np.random.seed(None)  # Náhodnost při každém spuštění
    
    # Generování dat
    X = np.linspace(0, 10, n_samples)
    y_true = np.sin(1.5 * X) + 0.1 * X
    noise = np.random.normal(0, noise_level, n_samples)
    y = y_true + noise
    
    # Náhodné odstranění dat
    if missing_data_percent > 0:
        n_missing = int(n_samples * missing_data_percent / 100)
        missing_indices = np.random.choice(n_samples, n_missing, replace=False)
        mask = np.ones(n_samples, dtype=bool)
        mask[missing_indices] = False
        X_train = X[mask].reshape(-1, 1)
        y_train = y[mask]
    else:
        X_train = X.reshape(-1, 1)
        y_train = y
    
    X_test = np.linspace(-1, 11, 300).reshape(-1, 1)
    
    # Výběr modelu
    if model_type == "Gaussovský proces":
        from sklearn.gaussian_process import GaussianProcessRegressor
        from sklearn.gaussian_process.kernels import RBF, WhiteKernel
        
        kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=noise_level)
        model = GaussianProcessRegressor(kernel=kernel, alpha=0.1)
        model.fit(X_train, y_train)
        y_pred, y_std = model.predict(X_test, return_std=True)
        
    elif model_type == "Random Forest":
        model = RandomForestRegressor(n_estimators=50, random_state=42)
        model.fit(X_train, y_train)
        
        # Získání nejistoty z ensemble
        predictions = np.array([tree.predict(X_test) for tree in model.estimators_])
        y_pred = np.mean(predictions, axis=0)
        y_std = np.std(predictions, axis=0)
        
    else:  # Bayesovská regrese
        model = BayesianRidge()
        model.fit(X_train, y_train)
        y_pred, y_std = model.predict(X_test, return_std=True)
    
    # Vizualizace
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Graf 1: Predikce s nejistotou
    ax1.scatter(X_train, y_train, c='red', s=50, alpha=0.7, 
               edgecolors='black', label='Trénovací data', zorder=5)
    ax1.plot(X_test, y_pred, 'b-', linewidth=2, label=f'{model_type} predikce')
    ax1.fill_between(X_test.ravel(), 
                    y_pred - 2*y_std, 
                    y_pred + 2*y_std, 
                    alpha=0.3, color='blue', label='95% interval spolehlivosti')
    
    # Označení extrapolace
    ax1.axvspan(-1, 0, alpha=0.1, color='orange', label='Extrapolace')
    ax1.axvspan(10, 11, alpha=0.1, color='orange')
    
    ax1.set_xlabel('X', fontsize=12)
    ax1.set_ylabel('Y', fontsize=12)
    ax1.set_title(f'{model_type} - Predikce s kvantifikovanou nejistotou', 
                 fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(-1, 11)
    
    # Graf 2: Mapa nejistoty
    ax2.plot(X_test, y_std, 'g-', linewidth=3, label='Standardní odchylka')
    ax2.fill_between(X_test.ravel(), 0, y_std, alpha=0.3, color='green')
    ax2.set_xlabel('X', fontsize=12)
    ax2.set_ylabel('Nejistota (σ)', fontsize=12)
    ax2.set_title('Prostorové rozdělení nejistoty', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(-1, 11)
    
    # Označení oblastí
    ax2.axvspan(-1, 0, alpha=0.1, color='orange')
    ax2.axvspan(10, 11, alpha=0.1, color='orange')
    
    plt.tight_layout()
    
    # Statistiky
    avg_uncertainty = np.mean(y_std)
    max_uncertainty = np.max(y_std)
    min_uncertainty = np.min(y_std)
    
    stats_text = f"""### Statistiky nejistoty:
    
**Model:** {model_type}
**Úroveň šumu:** {noise_level:.2f}
**Chybějící data:** {missing_data_percent:.0f}%
**Počet trénovacích vzorků:** {len(X_train)}

**Průměrná nejistota (σ):** {avg_uncertainty:.3f}
**Maximální nejistota:** {max_uncertainty:.3f}
**Minimální nejistota:** {min_uncertainty:.3f}

**Pozorování:**
- Nejistota se zvyšuje v oblastech s méně daty
- Extrapolace mimo trénovací oblast má vysokou nejistotu
- Různé modely kvantifikují nejistotu různě
"""
    
    return fig, stats_text

# Vytvoření Gradio rozhraní
with gr.Blocks(title="Simulátor nejistoty v AI") as demo:
    gr.Markdown("# 🎯 Simulátor nejistoty v AI modelech")
    gr.Markdown("""Experimentujte s různými faktory ovlivňujícími nejistotu v predikčních modelech.
    Pozorujte, jak se mění interval spolehlivosti v závislosti na:
    - Typu modelu
    - Úrovni šumu v datech
    - Množství chybějících dat
    - Počtu trénovacích vzorků""")
    
    with gr.Row():
        with gr.Column():
            model_choice = gr.Radio(
                choices=["Gaussovský proces", "Random Forest", "Bayesovská regrese"],
                value="Gaussovský proces",
                label="Typ modelu"
            )
            
            noise = gr.Slider(
                minimum=0,
                maximum=1,
                value=0.3,
                step=0.05,
                label="Úroveň šumu v datech"
            )
            
            missing = gr.Slider(
                minimum=0,
                maximum=70,
                value=20,
                step=5,
                label="Procento chybějících dat (%)"
            )
            
            samples = gr.Slider(
                minimum=20,
                maximum=200,
                value=50,
                step=10,
                label="Počet vzorků"
            )
            
            simulate_btn = gr.Button("🚀 Spustit simulaci", variant="primary")
        
        with gr.Column():
            stats_output = gr.Markdown("### Zde se zobrazí statistiky...")
    
    plot_output = gr.Plot(label="Vizualizace nejistoty")
    
    simulate_btn.click(
        uncertainty_simulator,
        inputs=[model_choice, noise, missing, samples],
        outputs=[plot_output, stats_output]
    )
    
    gr.Markdown("""### 📚 Vysvětlení modelů:
    
- **Gaussovský proces**: Neparametrický model s přirozenou kvantifikací nejistoty
- **Random Forest**: Ensemble stromů - nejistota z variance predikcí
- **Bayesovská regrese**: Pravděpodobnostní lineární model s posteriorním rozdělen
    """)

# Spuštění aplikace
demo.launch(share=True)

## 6. Shrnutí a klíčové koncepty

### Co jsme se naučili:

1. **Zdroje nejistoty v AI**
   - Aleatorická nejistota (přirozená náhodnost)
   - Epistemická nejistota (nedostatek znalostí)
   - Chybějící data a šum

2. **Pravděpodobnostní modely**
   - Poskytují nejen predikci, ale i míru jistoty
   - Umožňují kvantifikovat riziko
   - Důležité pro rozhodování v reálných aplikacích

3. **Bayesovo uvažování**
   - Aktualizace přesvědčení s novými daty
   - Kombinace apriorních znalostí s pozorováními
   - Sekvenční učení a adaptace

4. **Praktické metody**
   - Strategie pro práci s chybějícími daty
   - Ensemble metody pro odhad nejistoty
   - Bayesovské modely s přirozenou kvantifikací

### Klíčové principy:

- **Nejistota není chyba** - je to přirozená součást predikce
- **Kvantifikace nejistoty** umožňuje lepší rozhodování
- **Bayesovský přístup** poskytuje principiální framework
- **Různé modely** mají různé způsoby práce s nejistotou

## 7. Domácí úkol

### Úkol 1: Bayesovský detektor anomálií
Implementujte systém, který:
- Postupně se učí normální chování systému
- Detekuje anomálie s kvantifikací jistoty
- Aktualizuje své přesvědčení s novými daty
- Vizualizuje vývoj detekce v čase

### Úkol 2: Predikce s odmítnutím
Vytvořte klasifikátor, který:
- Může odmítnout klasifikaci při vysoké nejistotě
- Optimalizuje práh odmítnutí pro minimalizaci chyb
- Porovná výkon s různými prahy

### Úkol 3: Aktivní učení
Implementujte systém, který:
- Identifikuje vzorky s nejvyšší nejistotou
- Požádá o jejich označení (simulovaně)
- Ukáže, jak se zlepšuje s aktivním výběrem dat

### Bonusový úkol: Ensemble nejistoty
Vytvořte vlastní ensemble model, který:
- Kombinuje různé typy modelů
- Agreguje jejich nejistoty
- Poskytuje robustnější odhady

---

💡 **Tip**: Při řešení úkolů se zaměřte na praktickou interpretaci nejistoty a její využití pro lepší rozhodování!